In [3]:
import gradio as gr
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = tf.keras.models.load_model('turkce_sentiment_model.h5')
with open('tokenizer.pickle', 'rb') as h:
    tokenizer = pickle.load(h)
with open('label_encoder.pickle', 'rb') as h:
    le = pickle.load(h)

etiket_map = {
    "Positive": "Pozitif 😊",
    "Negative": "Negatif 😡",
    "Notr": "Nötr 😐" 
}

def duygu_analizi_yap(cumle):
    cumle = cumle.replace('İ', 'i').replace('I', 'ı').lower()
    seq = tokenizer.texts_to_sequences([cumle])
    padded = pad_sequences(seq, maxlen=100)
    pred = model.predict(padded, verbose=0)[0]
    
  
    sonuc_sozlugu = {}
    for i, olasilik in enumerate(pred):
        ing_etiket = le.classes_[i] 
        tr_etiket = etiket_map.get(ing_etiket, ing_etiket)
        sonuc_sozlugu[tr_etiket] = float(olasilik) 
        
    return sonuc_sozlugu

arayuz = gr.Interface(
    fn=duygu_analizi_yap, 
    inputs=gr.Textbox(lines=2, placeholder="Cümlenizi buraya yazın...", label="Metin Girişi"),
    outputs=gr.Label(num_top_classes=3, label="Duygu Analizi Güven Oranı"),
    title="Türkçe Metinlerde Duygu Analizi",
    description="LSTM modeli kullanarak metnin duygusunu ve modelin bu tahminden ne kadar emin olduğunu analiz edin.",
    theme=gr.themes.Soft(),
    examples=[
        "Çok açıklayıcı bir video, teşekkürler.", 
        "İzlemeyin. Gereksiz boş bir film. 2 saatinize yazık.", 
        "Kurtların alt türleri hakkında farklı görüşler vardır."
    ]
)

if __name__ == "__main__":
    arayuz.launch(share=True)

C:\Users\EMRE\anaconda3\Lib\site-packages\gradio\interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  super().__init__(


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://cae282a5ad5420bc56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
